# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import pandas as pd
from lxml import etree
import time

Load XML

In [2]:
xml_file_path = 'C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/20241031-gleif-concatenated-file-lei2.xml'
out_path = "C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/out"

In [3]:
namespace = {'lei': 'http://www.gleif.org/data/schema/leidata/2016'}

# Initialize a list to store the extracted data
data = []

try:  # Placeholder namespace, adjust if necessary

    # Use iterparse to read the XML file

    context = etree.iterparse(xml_file_path, events=('end',),
                                  tag='{http://www.gleif.org/data/schema/leidata/2016}LEIRecord')

    for event, lei_record in context:
        lei = lei_record.find('.//lei:LEI', namespaces=namespace).text
        legal_name = lei_record.find('.//lei:LegalName', namespaces=namespace).text
        legal_form_code = lei_record.find('.//lei:EntityLegalFormCode', namespaces=namespace)
        other_legal_form = lei_record.find('.//lei:OtherLegalForm', namespaces=namespace)

        # Use fallback values for legal form fields
        legal_form_code_text = legal_form_code.text if legal_form_code is not None else "N/A"
        other_legal_form_text = other_legal_form.text if other_legal_form is not None else "N/A"

        data.append({
            'Entity_Name': legal_name,
            'LEI': lei,
            'Legal_Code': legal_form_code_text,
            'Legal_Form': other_legal_form_text
        })

        # Clear the element to free memory
        lei_record.clear()

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
except FileNotFoundError:
    print(f"The file '{xml_file_path}' was not found.")
except etree.XMLSyntaxError:
    print("There was an error parsing the XML data.")
except Exception as e:
    print(f"An error occurred: {e}")

Merge with Legal Form

In [5]:
name_df = pd.read_csv("C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/2023-09-28-elf-code-list-v1.5.csv")
#rename for readability
name_df = name_df[["Entity Legal Form name Transliterated name (per ISO 01-140-10)", "ELF Code"]]
name_df.rename(columns={"Entity Legal Form name Transliterated name (per ISO 01-140-10)": 'Legal_Form', 'ELF Code': 'Legal_Code'}, inplace=True)

In [7]:
df = df.merge(name_df, on="Legal_Code", how="inner", suffixes=('', '_drop'))
df = df.loc[:, ~df.columns.str.endswith('_drop')]

Add industry and size

In [8]:
columns_to_load = ['name', 'industry', 'size'] # used due to memory error
file_path_company_dataset = 'C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/free_company_dataset.csv'
company_df = pd.read_csv(file_path_company_dataset, on_bad_lines='skip', usecols=columns_to_load) # bad line at 37k+
company_df.rename(columns={"name": 'Entity_Name',}, inplace=True) #rename for readability

In [10]:
df = df.merge(company_df, on="Entity_Name", how="left", suffixes=('', '_drop'))
df.to_csv(out_path + '/output.csv', sep=';', index=False)

In [11]:
df_reduced = df[df["industry"].notna()]
df_reduced = df_reduced[df_reduced["Legal_Form"] != 'N/A']
df_reduced.to_csv(out_path + '/reduced_output.csv', sep=';', index=False)

#1 Drop Suffixes 

In [12]:
forward_df = pd.read_excel("C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/forward_firm_universe.xlsx")


In [13]:
# Extract the last word from each entry in 'Entity_Name' as a potential suffix
forward_df['Suffix'] = forward_df['Entity_Name'].apply(lambda x: x.split(" ")[-1])

suffix_list = forward_df['Suffix'].unique()

with open(out_path+"/suffixes.txt", "w",  encoding="utf-8") as file:
    for item in suffix_list:
        file.write(f"{item}\n",)

In [14]:

# List of suffixes to remove
suffixes = [
    "PLC", "Co", "Corp", "Ltd", "LLC", "OOO", "PJSC", "SA", "SAE", "CJSC", "SAOG", 
    "SAOC", "SpA", "Bhd", "SAS", "Inc", "CV", "JSC", "Ltda", "Ltd.", "PAO", "AS", 
    "AŞ", "OAO", "AG", "LP", "LLP", "Co.,Ltd.", "Co.", "LTD", "BV", "PJS", "SAA", 
    "SPA", "AO", "S.A.", "ULC", "CVBA", "GmbH", "S.p.A.", "PCL", "Ltd.Shanxi", 
    "TOO", "B.V", "ShA", "AD", "PrAT", "PSC", "EAD", "Srl", "JV", "PT", "AB", 
    "S.A", "SARL", "Tbk", "S.L.", "Llc", "Limited", "LLC.", "PSA", "C.V.", 
    "P/S", "Ltd.LLC", "P.L.C.", "INC.", "KgaA", "NV", "Pvt", "OC", "SA/NV", 
    "JSC", "QSC", "QPSC", "LLC", "S.A.S.", "Cooperativa", "ASBL", "A/S", "SPV", 
    "Ltd(Sarl)", "AD(SA)", "Holdings", "D.O.O.", "KG", "ZRT", "A/S", "N.V.", 
    "CSJC", "LLC(Ltd)", "Sociedad", "SA(SP)", "GmbH & Co. KG", "AG & Co. KGaA", 
    "S.A.C.", "S.C.A.", "Cie", "GIE", "Co.,Ltd", "Spółka", "OOO", "KFT", "SNC", 
    "SAIC", "S.A.I.C.", "EEIG", "WLL", "Inc.", "Ltd/Ltd", "Sdn Bhd", "JLL", 
    "Plc", "Npo", "CO.LTD", "Ltda.", "I/S", "Inc..", "V.o.S.", "v.o.f.", "d.o.o.", 
    "株式会社", "KK", "Pty Ltd", "IAC", "S.A.P.I. de C.V.", "D.I.O.", "CBO", 
    "Comp.", "C.A", "B.V.", "Ltd.Pvt.", "Ltd LLC", "Ltd LLC.", "Pvt.Ltd.", 
    "S/A", "S.L", "A.E.I.E.", "GmbH", "OÜ", "S.R.L.", "W.L.L.", "Coop.", "B.V.I.", 
    "ltd.", "Ltd(Pty)", "有限会社", "주식회사", "DMCC", "CNSX", "Pty", "EIRL", "Comp.", 
    "S.A.R.L.", "ltd", "株式会社", "C.A.", "OÜ", "B.V.", "D.O.O.", "CJSC", "PJSC", 
    "Yhtiö", "株式会社", "Ltd", "株式会社", "EPE", "Csp", "GIE", "JLL", "Ltd(Company)", 
    "S.A.P.I.", "V.O.S.", "Società", "합자회사", "株式会社", "B.V.", "SARL", 
    "股份有限公司", "KT", "Pty.", "Co., Ltd.", "N.V.", "S.R.O.", "L.C.", "C.B.", "AB", 
    "L.L.C", "S.R.L.", "株式会社", "K.K.", "Pty Ltd", "L.P.", "AO", "P.C.", "A/S", 
    "I/K", "株式会社", "Ltd", "股份有限公司", "A/S", "株式会社"
]


# Create a regex pattern that matches any suffix at the end of the string
pattern = r'\b(' + '|'.join(suffixes) + r')\b\.?$'
print(f"prev size:{len(forward_df)}")
# Remove suffixes from "Entity_Name" column
forward_df['Entity_Name'] = forward_df['Entity_Name'].str.replace(pattern, '', regex=True).str.strip()
forward_df.drop_duplicates(subset=['Entity_Name'], inplace=True)
print(f"after reduction size:{len(forward_df)}")

prev size:23446
after reduction size:23043


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein
from fuzzywuzzy import fuzz

# Function to calculate cosine similarity between two strings
def cosine_sim(text1, text2):
    try:
        vectorizer = TfidfVectorizer(stop_words=None).fit_transform([text1, text2])
        cosine_similarities = cosine_similarity(vectorizer[0:1], vectorizer[1:2])
        return cosine_similarities[0][0]
    except Exception as e:
        print(e)
        return 0


# Function to calculate Levenshtein distance between two strings
def levenshtein_dist(text1, text2):
    return Levenshtein.distance(text1, text2)


# Function to calculate FuzzyWuzzy ratio between two strings
def fuzzywuzzy_ratio(text1, text2):
    return fuzz.ratio(text1, text2)


# Function to merge similar names based on a selected method
def merge_similar_names(df, name, method='cosine', cosine_threshold=0.8, lev_threshold=2, fuzzy_threshold=85):
    # Define similarity functions based on the selected method
    if method == 'cosine':
        similarity_func = cosine_sim
        threshold_func = lambda score: score >= cosine_threshold and score != 1
    elif method == 'levenshtein':
        similarity_func = levenshtein_dist
        threshold_func = lambda score: score <= lev_threshold and score != 0
    elif method == 'fuzzywuzzy':
        similarity_func = fuzzywuzzy_ratio
        threshold_func = lambda score: score >= fuzzy_threshold and score != 100
    else:
        raise ValueError("Invalid method selected. Choose from 'cosine', 'levenshtein', or 'fuzzywuzzy'.")

    # Iterate over all company names
    df['score'] = df['Entity_Name'].apply(similarity_func, args=(name,))
    df['score'] = df['score'].apply(threshold_func)
    return df

Levenshtein

In [19]:
#methods = ['levenshtein', 'fuzzywuzzy', 'cosine']
forward_df = pd.read_excel("C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/forward_firm_universe.xlsx")

forward_df['prefix'] = forward_df['Entity_Name'].apply(lambda x: x.split(" ")[0])
prefix_list = list(forward_df['prefix'].unique())
forward_df['score'] = False
selected_method = 'levenshtein'
print(selected_method)
print(f"prev size:{len(forward_df)}")
start_time = time.time()
for prefix in prefix_list:
    #print(f"{prefix_list.index(prefix) + 1}/{len(prefix_list)}")
    working_df = forward_df[forward_df['Entity_Name'].str.startswith(prefix)]
    if len(working_df) < 2:
        continue
    # selected_method = 'cosine' # Change this to 'levenshtein' or 'fuzzywuzzy' to use other methods
    merged_df = merge_similar_names(working_df.copy(), prefix, method=selected_method, cosine_threshold=0.7,
                                    lev_threshold=2,
                                    fuzzy_threshold=85)
    forward_df.update(merged_df)

forward_df = forward_df[~forward_df['score']]
end_time = time.time()
# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")
print(f"after reduction size:{len(forward_df)}")


levenshtein
prev size:23446
Time taken: 115.815526 seconds
after reduction size:23414


fuzzywuzzy

In [20]:
forward_df = pd.read_excel("C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/forward_firm_universe.xlsx")

#methods = ['levenshtein', 'fuzzywuzzy', 'cosine']
forward_df['prefix'] = forward_df['Entity_Name'].apply(lambda x: x.split(" ")[0])
prefix_list = list(forward_df['prefix'].unique())
forward_df['score'] = False
selected_method = 'fuzzywuzzy'
print(selected_method)
print(f"prev size:{len(forward_df)}")
start_time = time.time()
for prefix in prefix_list:
    #print(f"{prefix_list.index(prefix) + 1}/{len(prefix_list)}")
    working_df = forward_df[forward_df['Entity_Name'].str.startswith(prefix)]
    if len(working_df) < 2:
        continue
    # selected_method = 'cosine' # Change this to 'levenshtein' or 'fuzzywuzzy' to use other methods
    merged_df = merge_similar_names(working_df.copy(), prefix, method=selected_method, cosine_threshold=0.7,
                                    lev_threshold=2,
                                    fuzzy_threshold=85)
    forward_df.update(merged_df)

forward_df = forward_df[~forward_df['score']]
end_time = time.time()
# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")
print(f"after reduction size:{len(forward_df)}")


fuzzywuzzy
prev size:23446
Time taken: 116.896533 seconds
after reduction size:23420


cosine

In [21]:
forward_df = pd.read_excel("C:/Users/max00/Documents/Problem_Set_Forward Analytics/Problem_Set_Forward_Analytics/data/forward_firm_universe.xlsx")

#methods = ['levenshtein', 'fuzzywuzzy', 'cosine']
forward_df['prefix'] = forward_df['Entity_Name'].apply(lambda x: x.split(" ")[0])
prefix_list = list(forward_df['prefix'].unique())
forward_df['score'] = False
selected_method = 'cosine'
print(selected_method)
print(f"prev size:{len(forward_df)}")
start_time = time.time()
for prefix in prefix_list:
    #print(f"{prefix_list.index(prefix) + 1}/{len(prefix_list)}")
    working_df = forward_df[forward_df['Entity_Name'].str.startswith(prefix)]
    if len(working_df) < 2:
        continue
    # selected_method = 'cosine' # Change this to 'levenshtein' or 'fuzzywuzzy' to use other methods
    merged_df = merge_similar_names(working_df.copy(), prefix, method=selected_method, cosine_threshold=0.7,
                                    lev_threshold=2,
                                    fuzzy_threshold=85)
    forward_df.update(merged_df)

forward_df = forward_df[~forward_df['score']]
end_time = time.time()
# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")
print(f"after reduction size:{len(forward_df)}")


cosine
prev size:23446
empty vocabulary; perhaps the documents only contain stop words
Time taken: 200.539974 seconds
after reduction size:23425
